In [1]:
# Install Python dependencies
!pip install wikibaseintegrator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.9 MB/s eta 0:00:00


In [2]:
# A class for writing Entity claims to Wikidata

import textwrap

from wikibaseintegrator import WikibaseIntegrator, wbi_login
from wikibaseintegrator.datatypes import ExternalID, Item, MonolingualText, Time
from wikibaseintegrator.wbi_config import config as wbi_config
from wikibaseintegrator.wbi_enums import ActionIfExists
from wikibaseintegrator.wbi_exceptions import MWApiError
from wikibaseintegrator.wbi_helpers import search_entities


class WikidataCreateException(Exception):
    """
    This exception is raised when a Wikidata entity already exists so a new one can't be created.
    """
    pass


class WikidataWriteException(Exception):
    """
    This exception is raised when the entity/resource/xos_id isn't set.
    """
    pass


class Wikidata:
    """
    Writes ACMI IDs back to Wikidata.
    """

    def __init__(self, username, password):
        self.mediawiki_api_url = 'https://www.wikidata.org/w/api.php'
        self.login = wbi_login.Login(
            user=username,
            password=password,
            mediawiki_api_url=self.mediawiki_api_url,
        )
        wbi_config['USER_AGENT'] = 'ACMI_XOS/1.0 (https://www.wikidata.org/wiki/User:ACMI_Simon)'
        self.wikidata_writer = WikibaseIntegrator(login=self.login)

    def add_acmi_id(self, resource, xos_id, wikidata_id, summary=None):
        """
        Add or update an ACMI XOS ID to Wikidata.

        :param str resource: Should be set to 'works' or 'creators'
        :param str xos_id: The XOS ID that can be found in an ACMI Website URL: https://www.acmi.net.au/works/117993
        :param str wikidata_id: The Wikidata entity ID to update. e.g. Q123456
        """
        success = False

        if not resource or not xos_id or not wikidata_id:
            raise WikidataWriteException(
                'To write to Wikidata you must set a resource (works/creators), an xos_id, and a Wikidata ID.',
            )

        if not summary:
            summary = 'Added ACMI public identifier.'

        item = self.wikidata_writer.item.get(
            wikidata_id,
            mediawiki_api_url=self.mediawiki_api_url,
            login=self.login,
        )

        acmi_id = f'{resource}/{xos_id}'
        claim = ExternalID(prop_nr='P7003', value=acmi_id)
        item.claims.add(claim, action_if_exists=ActionIfExists.APPEND_OR_REPLACE)
        try:
            item.write(summary=summary)
            success = True
            print(f'{wikidata_id} ACMI ID added: {acmi_id}')
        except MWApiError as exception:
            print(f'Failed to add ACMI ID: {exception}')

        return success

    def create(self, resource, force=False):
        """
        Create a new Wikidata entry for this XOS resource.

        :param Work/Creator resource: An XOS Work or Creator class object.
        :param bool force: Whether to create a Wikidata entity without first checking one exists.
        """
        title = ''
        description = ''
        language = 'en'
        resource_type = resource.__class__.__name__.lower()
        success = False

        if resource_type == 'work':
            title = resource.get_title_display()
            description = self.build_description(resource)
        elif resource_type == 'creator':
            title = resource.name
            description = resource.biography

        description = textwrap.shorten(description, width=250, placeholder='...')

        if not title or not description:
            raise WikidataCreateException(
                'Please add a title/name and description/biography before creating a Wikidata entry',
            )

        wikidata_reference = resource.external_references.filter(source__slug='wikidata').first()
        if wikidata_reference:
            raise WikidataCreateException(
                f'A Wikidata external reference already exists: {wikidata_reference.source_identifier}',
            )

        wikidata_entity = search_entities(title)
        if wikidata_entity and not force:
            raise WikidataCreateException(
                f'A Wikidata entity already exists: {",".join(wikidata_entity)}',
            )

        item = self.wikidata_writer.item.new()
        item.labels.set(language=language, value=title)
        item.descriptions.set(language=language, value=description)
        title_statement = MonolingualText(text=title, language=language, prop_nr='P1476')
        item.title = title_statement

        if resource_type == 'work':
            # Add instance of work type
            instance_of = self.build_work_type_claim(resource)
            if instance_of:
                item.claims.add(instance_of)

            if resource.production_dates.first():
                # Publication year
                year = resource.production_dates.first().to_year()
                if year:
                    # Remove non-digits
                    year = ''.join(filter(str.isdigit, year))
                    date_statement = Time(
                        time=f'+{year}-01-01T00:00:00Z',
                        prop_nr='P577',
                        precision=9,
                    )
                    item.claims.add(date_statement)

        if resource_type == 'creator':
            role_names = [role_in_work.role.name for role_in_work in resource.roles_in_work.all()]
            if not any('company' in role_name for role_name in role_names):
                # Person
                instance_of_person = Item(value='Q5', prop_nr='P31')
                item.claims.add(instance_of_person)
                name_statement = MonolingualText(text=title, language=language, prop_nr='P1559')
                item.claims.add(name_statement)

        acmi_id = f'{resource_type}s/{resource.id}'
        claim = ExternalID(prop_nr='P7003', value=acmi_id)
        item.claims.add(claim)

        try:
            wikidata_response = item.write()
            print(f'ACMI {resource_type} created: {resource}')
            if wikidata_response.id:
                success = wikidata_response.id
                print(f'Wikidata ID: {wikidata_response.id}')
        except MWApiError as exception:
            print(f'Failed to create {resource}: {exception}')

        return success

    def build_description(self, work):
        """
        Build a Wikidata standard description for this Work.
        """
        description = ''
        description_list = []
        role_names = [
            'artist',
            'creator',
            'director',
        ]
        year = work.get_first_production_year()
        if year:
            description_list.append(year)
        if year and work.work_type:
            description_list.append(work.work_type.lower())
        elif work.work_type:
            description_list.append(work.work_type)
        creators = work.top_names
        for creator in creators:
            if creator.roles_in_work.filter(work=work, role__name__in=role_names):
                description_list.append(f'by {creator.name}')
        if description_list:
            description = ' '.join(description_list)
        return description

    def build_work_type_claim(self, work):
        """
        Build a Wikidata claim for this type of Work.
        """
        claim = None
        if work.work_type == Work.WORK_TYPE_CHOICES[0][0]:
            # Film
            claim = Item(value='Q11424', prop_nr='P31')

        if work.work_type == Work.WORK_TYPE_CHOICES[1][0]:
            # TV Show
            claim = Item(value='Q5398426', prop_nr='P31')

        if work.work_type == Work.WORK_TYPE_CHOICES[2][0]:
            # Videogame
            claim = Item(value='Q7889', prop_nr='P31')

        if work.work_type == Work.WORK_TYPE_CHOICES[3][0]:
            # Object
            claim = Item(value='Q488383', prop_nr='P31')

        if work.work_type == Work.WORK_TYPE_CHOICES[4][0]:
            # Image
            claim = Item(value='Q478798', prop_nr='P31')

        if work.work_type == Work.WORK_TYPE_CHOICES[5][0]:
            # Music video
            claim = Item(value='Q193977', prop_nr='P31')

        if work.work_type == Work.WORK_TYPE_CHOICES[6][0]:
            # Video
            claim = Item(value='Q98069877', prop_nr='P31')

        if work.work_type == Work.WORK_TYPE_CHOICES[8][0]:
            # Artwork
            claim = Item(value='Q735', prop_nr='P31')

        if work.work_type == Work.WORK_TYPE_CHOICES[9][0]:
            # Mixed reality
            claim = Item(value='Q1758389', prop_nr='P31')

        return claim